In [1]:
from lxml import etree

In [22]:
college = 'Algonquin'

infile = 'input/' + college +'_invoices_20211123.xml'
outfile = 'output/' + college +'_invoices_fixed_20211124.xml'

In [25]:
# Specify the XMLParser with recover option to catch any '&' that 
# have not been properly escaped in the source file.
tree = etree.parse(infile, parser=etree.XMLParser(recover=True))


In [26]:
# Start by setting all currency attributes to CAD
currencies = tree.xpath('//*[@currency]')
for currency in currencies:
    currency.attrib['currency']='CAD'

In [27]:
# Where exchangeRate is present, distinguish between native amount and converted amount and specify currency
converted_amounts = tree.xpath('//exchangeRate')
print(len(converted_amounts))

for amount in converted_amounts:
    
    native_amounts = amount.xpath('../../..//nativeAmount|../../..//nativeAmountPaid|../../..//amountPaid')
    #print(len(native_amounts))
    for native in native_amounts:
        native.attrib['currency']='USD'
    
    converted_amounts = amount.xpath('../../..//vendorAmount|../../..//vendorAmountPaid|../../..//vendorFinalPrice|../../..//remitToVendor')
    for converted in converted_amounts:
        converted.attrib['currency']='CAD'
    
tree.write(outfile, pretty_print=True)

61
